### Test Here

In [ ]:
import sys
sys.path.insert(0,'C:/Users/owner/Documents/Github/gym-warehouse')

import gym
import gym_warehouse
import numpy as np
import copy

env = gym.make('warehouse-v0')
env.reset()
env.render()

In [ ]:
# CNNDQN
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# torch.manual_seed(1356)

# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# torch.cuda.manual_seed(1356)
# torch.backends.cudnn.deterministic = True

# out_dir = '/u/training/tra442/scratch/'
out_dir = 'outdir4/'

# from wrappers import make_atari, wrap_deepmind, wrap_pytorch
# env    = make_atari(env_id)
# env    = wrap_deepmind(env)
# env    = wrap_pytorch(env)


import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import matplotlib.pyplot as plt

USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action  = q_value.max(1)[1].data.item()
        else:
            action = random.randrange(25)
        return action

def plot(frame_idx, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.savefig(out_dir + 'rewards.png')

    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.savefig(out_dir + 'losses.png')
    plt.show()
    

In [ ]:

current_model = torch.load(out_dir +'current.model')
current_model.load_state_dict(torch.load(out_dir + 'current.ckpt'))

target_model = torch.load(out_dir + 'target.model')
target_model.load_state_dict(torch.load(out_dir + 'target.ckpt'))


if USE_CUDA:
    current_model = current_model.cuda()
    target_model  = target_model.cuda()


num_frames = 50000000


losses = []
all_rewards = []
episode_reward = 0

state = env.reset()
state = state.flatten()
# env.render()
for frame_idx in range(1, num_frames + 1):
    epsilon = 0
    action = current_model.act(state, epsilon)
    
    print(action)

    next_state, reward, done, _ = env.step(action)
    next_state=next_state.flatten()
    
    env.render()

    state = next_state
    episode_reward += reward[0]+reward[1]


    if done:
        state = env.reset()
        state = state.flatten()
        all_rewards.append(episode_reward)
        print("Episode Reward When Done: {}".format(episode_reward))
        episode_reward = 0


    if frame_idx % 10000==0:
        print('Frame: ',frame_idx)
        print("Episode Reward: {}".format(episode_reward))